### This code is based on <a href="https://www.digitalocean.com/community/tutorials/how-to-use-the-sqlite3-module-in-python-3">How To Use the sqlite3 Module in Python 3</a>

This tutorial simulates an inventory of fish that need to be modified as fish are added to or removed from a fictional aquarium.

The main goal of this tutorial is to show how to create a connection to a SQLite database, add a table to that database, insert data into that table, and read and modify data in that table.

In [1]:
# importing necessary libraries and modules
import sqlite3

In [2]:
# checking SQLite version - just for demonstration purposes
sqlite3.sqlite_version

'3.33.0'

### Step 1 - establishing a connection to a database

The *sqlite3.connect()* function returns a <a href="https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection">Connection</a> object that we will use to interact with the SQLite database held in the file ``aquarium.db``. The ``aquarium.db`` file is created automatically by sqlite3.connect() if it does not already exist.

In [3]:
# creating a connection to the database file named ``aquarium.db``. 
# Alternatively, you can create a database entirely hosted in memory by using sqlite3.connect(":memory:").
# This database will be deleted when the program exits.
connection = sqlite3.connect("aquarium.db")

*connection.total_changes* is the total number of database rows that have been changed by the connection. Since we have not executed any SQL commands yet, ``0 total_changes`` is correct.

In [4]:
print(connection.total_changes)

0


### Step 2 - creating tables and inserting data

In a SQL database, data is stored in tables. Tables define a set of columns, and contain 0 or more rows with data for each of the defined columns.

We will create a table named ``fish`` that tracks a value for *name, species,* and *tank_number* for each fish at the aquarium. Two example fish rows are listed: one row for a shark named Sammy, and one row for a cuttlefish named Jamie.

| name | species | tank_number |
|:-----:|:-----:|:-----:|
| Sammy	| shark	| 1 |
| Jamie	| cuttlefish | 7 |


*connection.cursor()* returns a Cursor object. Cursor objects allow us to send SQL statements to a SQLite database using *cursor.execute()*. The "CREATE TABLE fish ..." string is a SQL statement that creates a table named ``fish`` with the three columns described earlier: *name* of type TEXT, *species* of type TEXT, and *tank_number* of type INTEGER.

In [5]:
# opening a cursor to the database...
cursor = connection.cursor()
# ...and creating a table using cursor.execute() - this command executes only one SQL statement at time.
cursor.execute("CREATE TABLE fish (name TEXT, species TEXT, tank_number INTEGER)")

In [6]:
# inserting data into table
cursor.execute("INSERT INTO fish VALUES ('Frankie', 'Shark', 1)")
cursor.execute("INSERT INTO fish VALUES ('Jamie', 'Cuttlefish', 7)")

# alternatively, we can use executescript() to run multiple SQL statements at once
cursor.executescript("""
    INSERT INTO fish VALUES ('Nemo', 'Clownfish', 2);
    INSERT INTO fish VALUES ('Dory', 'Regal Blue Tang', 2);
    INSERT INTO fish VALUES ('Oscar', 'Bluestreak Cleaner Wrasse', 1);
    """)

### Step 3 - reading data from the database

The *cursor.execute()* function runs a SELECT statement to retrieve values from the ``fish table``. 

*fetchall()* retrieves all the results of the SELECT statement. When we ``print(rows)``, we see the list of tuples retrieved from the database. 

In [7]:
# querying and retrieving data from the database
rows = cursor.execute("SELECT name, species, tank_number FROM fish").fetchall()
print(rows)

[('Frankie', 'Shark', 1), ('Jamie', 'Cuttlefish', 7), ('Nemo', 'Clownfish', 2), ('Dory', 'Regal Blue Tang', 2), ('Oscar', 'Bluestreak Cleaner Wrasse', 1)]


If we wanted to retrieve rows in the ``fish`` table that match a specific set of criteria, we can use a ``WHERE`` clause:

In [8]:
# using WHERE clause to match a specific criteria
# the placeholder (?) is used to substitute the target_fish_name variable into the SELECT statement.
target_fish_name = "Jamie"
rows = cursor.execute(
    "SELECT name, species, tank_number FROM fish WHERE name = ?",
    (target_fish_name,),
).fetchall()
print(rows)

[('Jamie', 'Cuttlefish', 7)]


### Step 4 - modifying and removing data in the database

Rows in a SQLite database can be modified using *UPDATE* and *DELETE* SQL statements.

In [9]:
# using an UPDATE SQL statement to change the tank_number of Frankie to its new value of 2. 
# The WHERE clause in the UPDATE statement ensures we only change the value of tank_number if a row has name = "Frankie".
new_tank_number = 2
moved_fish_name = "Frankie"
cursor.execute(
    "UPDATE fish SET tank_number = ? WHERE name = ?",
    (new_tank_number, moved_fish_name)
)

In [10]:
# checking if the UPDATE was made correctly
rows = cursor.execute("SELECT name, species, tank_number FROM fish").fetchall()
print(rows)

[('Frankie', 'Shark', 2), ('Jamie', 'Cuttlefish', 7), ('Nemo', 'Clownfish', 2), ('Dory', 'Regal Blue Tang', 2), ('Oscar', 'Bluestreak Cleaner Wrasse', 1)]


In [11]:
# issuing a DELETE SQL statement to remove a row.
# e.g. we have released Dory into wild 
released_fish_name = "Dory"
cursor.execute(
    "DELETE FROM fish WHERE name = ?",
    (released_fish_name,)
)

In [12]:
# checking if the DELETE was made correctly
rows = cursor.execute("SELECT name, species, tank_number FROM fish").fetchall()
print(rows)

[('Frankie', 'Shark', 2), ('Jamie', 'Cuttlefish', 7), ('Nemo', 'Clownfish', 2), ('Oscar', 'Bluestreak Cleaner Wrasse', 1)]


### Step 5 — using ``with`` statements for automatic cleanup

In this tutorial, we’ve used two primary objects to interact with the ``aquarium.db`` SQLite database: a Connection object named *connection*, and a Cursor object named *cursor*. These objects should be closed when they are no longer needed. We can use a ``with`` statement to help us automatically close them.



In [13]:
from contextlib import closing

# closing the Connection and Cursor objects
with closing(sqlite3.connect("aquarium.db")) as connection:
    with closing(connection.cursor()) as cursor:
        rows = cursor.execute("SELECT 1").fetchall()
        print(rows)

[(1,)]


``closing`` is a convenience function provided by the <a href="https://docs.python.org/3/library/contextlib.html">contextlib module</a>. When a ``with`` statement exits, ``closing`` ensures that *close()* is called on whatever object is passed to it. The closing function is used twice in this example. Once to ensure that the Connection object returned by *sqlite3.connect()* is automatically closed, and a second time to ensure that the Cursor object returned by *connection.cursor()* is automatically closed.

Since ``SELECT 1`` is a SQL statement that always returns a single row with a single column with a value of 1, it makes sense to see a single tuple with 1 as its only value returned by our code.